# Preparation

In [64]:
from utils.data_prep import get_data
import plotly.graph_objects as go
import pandas as pd

In [65]:
df = get_data()


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12136 entries, 0 to 12135
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Document Date              12136 non-null  datetime64[ns]
 1   Year/Month                 12136 non-null  period[M]     
 2   Year                       12136 non-null  int64         
 3   Month                      12136 non-null  int64         
 4   Supplier Delivery Date     12136 non-null  datetime64[ns]
 5   Delivery Date              12136 non-null  datetime64[ns]
 6   Company Code               12136 non-null  int64         
 7   Country                    12136 non-null  object        
 8   Purchasing Doc.            12136 non-null  int64         
 9   Item                       12136 non-null  int64         
 10  Purchasing Org.            12136 non-null  int64         
 11  Plant                      12136 non-null  int64         
 12  Supp

In [67]:
df.describe()

,Year,Month,Company Code,Purchasing Doc.,Item,Purchasing Org.,Plant,Supplier,Material,Net Price,Ordered Quantity,Delivered Quantity,Open Quantity,Delivery Deviation (Days),Deviation Cause,Net Value,Counter
count,12136.000000,12136.000000,12136.000000,1.213600e+04,12136.000000,12136.000000,12136.000000,1.213600e+04,1.213600e+04,12136.00000,12136.000000,12136.000000,12136.000000,12136.000000,12136.000000,12136.000000,12136.0
mean,2019.345831,5.870798,53.845089,8.202713e+09,2.263019,5198.619644,418.523401,5.856480e+06,5.971887e+07,59.11504,4788.483846,4785.820119,2.705339,0.049852,0.108438,8221.603334,1.0
std,0.475658,3.569839,0.492278,1.761377e+08,5.309245,1001.337060,1418.333240,1.257628e+07,1.877953e+07,310.12142,25327.854342,25313.664528,143.925618,1.032111,0.836305,38865.009635,0.0
min,2019.000000,1.000000,52.000000,1.500090e+09,1.000000,54.000000,51.000000,1.028900e+04,1.000348e+07,1.00000,0.250000,0.000000,-200.000000,-5.000000,0.000000,0.250000,1.0
25%,2019.000000,3.000000,54.000000,8.207604e+09,1.000000,5400.000000,51.000000,5.333800e+04,5.188012e+07,1.25000,13.000000,13.000000,0.000000,0.000000,0.000000,150.800000,1.0
50%,2019.000000,5.000000,54.000000,8.207793e+09,1.000000,5400.000000,51.000000,4.328630e+05,5.360295e+07,1.77000,200.000000,200.000000,0.000000,0.000000,0.000000,578.000000,1.0
75%,2020.000000,9.000000,54.000000,8.207941e+09,1.000000,5410.000000,51.000000,5.044180e+05,7.039588e+07,2.63000,1000.000000,1000.000000,0.000000,0.000000,0.000000,2110.000000,1.0
max,2020.000000,12.000000,54.000000,8.208073e+09,123.000000,5420.000000,9699.000000,3.432722e+07,9.494030e+07,6895.49000,590000.000000,590000.000000,10000.000000,75.000000,10.000000,612300.000000,1.0


In [68]:
df.nunique()

Document Date                 438
Year/Month                     18
Year                            2
Month                          12
Supplier Delivery Date        470
Delivery Date                 472
Company Code                    3
Country                         3
Purchasing Doc.              9926
Item                           63
Purchasing Org.                 8
Plant                           9
Supplier                      166
Supplier Name                 160
Postal Code                   157
Street                        163
City                          147
Supplier Country               12
Material                     3014
Material Short Text          3011
Material Group                164
Material Group Text           157
Order Unit                      6
Net Price                     297
Ordered Quantity              954
Delivered Quantity            956
Open Quantity                  23
Delivery Deviation (Days)      21
Deviation Indicator             4
Deviation Caus

In [69]:
df.head()

,Document Date,Year/Month,Year,Month,Supplier Delivery Date,Delivery Date,Company Code,Country,Purchasing Doc.,Item,...,Ordered Quantity,Delivered Quantity,Open Quantity,Delivery Deviation (Days),Deviation Indicator,Deviation Cause,Deviation Cause Text,Net Value,Local Currency,Counter
0,2019-02-28,2019-02,2019,2,2019-03-07,2019-03-07,52,AT,8207406178,1,...,10.0,7.0,3,0,in time,3,under-delivery,6720.0,EUR,1
1,2019-02-28,2019-02,2019,2,2019-03-07,2019-03-07,52,AT,8207406178,2,...,10.0,7.0,3,0,in time,3,under-delivery,18162.0,EUR,1
2,2019-02-28,2019-02,2019,2,2019-03-07,2019-03-07,52,AT,8207406178,3,...,10.0,7.0,3,0,in time,3,under-delivery,5710.0,EUR,1
3,2019-02-28,2019-02,2019,2,2019-03-07,2019-03-07,52,AT,8207406178,4,...,10.0,7.0,3,0,in time,3,under-delivery,29890.0,EUR,1
4,2019-02-28,2019-02,2019,2,2019-03-07,2019-03-07,52,AT,8207406178,5,...,13.0,10.0,3,0,in time,3,under-delivery,9646.0,EUR,1


# Ordered Spend

## Numeric Point Charts

In [70]:
df_point_charts = df.groupby('Year').agg({
    'Document Date': 'count',
    'Net Value': 'sum'
}).reset_index().rename(columns={'Net Value': 'Ordered Spend', 'Document Date': 'Number of Orders'})


In [71]:
def os_numeric_point_chart(df, last_year = False):
    if last_year:
        year = 2019
    else:
        year = 2020
        
    df = df.loc[df['Year'] == year]

    ordered_spend = df['Ordered Spend'].iloc[0]
    number_of_orders = df['Number of Orders'].iloc[0]
    fig = go.Figure()
    fig.add_trace(
        go.Indicator(mode="number", value=ordered_spend, domain={
            'x': [0, 0.45],
            'y': [0, 1]
            },
            title='Ordered Spend')) 
    fig.add_trace(
        go.Indicator(mode="number", value=number_of_orders, domain={
            'x': [0.55, 1],
            'y': [0, 1]
        }, title='Number of Orders'))
    fig.update_layout(title_text=year)
    return fig

### This Year

In [72]:
os_numeric_point_chart(df_point_charts).show()

### Last Year

In [73]:
os_numeric_point_chart(df_point_charts, True).show()

## Line Charts

In [74]:
df_line_charts = df.groupby(['Year', 'Month']).agg({
    'Document Date': 'count',
    'Net Value': 'sum'
}).reset_index().rename(columns={
    'Net Value': 'Ordered Spend',
    'Document Date': 'Number of Orders'
})

In [75]:
def os_line_chart(df, number_of_orders=False):
    if number_of_orders:
        displayed = 'Number of Orders'
    else:
        displayed = 'Ordered Spend'

    df_this_year = df.loc[df['Year'] == 2020]
    df_last_year = df.loc[df['Year'] == 2019]
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_this_year['Month'], y=df_this_year[displayed], mode='lines+markers', name=2020))
    fig.add_trace(go.Scatter(x=df_last_year['Month'], y=df_last_year[displayed], mode='lines+markers', name=2019))
    fig.update_layout(title_text=displayed)

    return fig

### Ordered Spend by Month

In [76]:
os_line_chart(df_line_charts).show()

KeyError: 'Ordered Spend'

### Number of Orders by Month

In [ ]:
os_line_chart(df_line_charts, True).show()

## Pie Charts

### Ordered Spend by Organisation

### Number of Orders by Organisation

## Bar Chart

### Top 10 Suppliers

# Supplier Performance